In [1]:
%run '/mnt/berger2/equipes/IGE/meom/workdir/berger2/fonctions/meom_fonctions/fonctions.ipynb'

import numpy as np
import matplotlib.pyplot as plt
import scipy.io.netcdf as netcdf
import qgutils as qg
from matplotlib import ticker, cm
plt.ion()

In [34]:
#paramètres gravés dans le marbre pour le stage

Delta = 10000                               #grid spacing
f0 = 9.37456E-05                           #paramètre de Coriolis
dh = np.array([350,750,2900])              #hauteurs des couches océaniques
dh_b = np.array([(dh[0]+dh[1])/2,(dh[0]+dh[1])/2])
g_prime = np.array([2.5E-02,1.25E-02])     #gravités réduites aux interfaces
Beta = 1.75360E-11
delta_ek = 5                               #ocean bottom Ekman thickness 
nu4 = 2E09                                 #bi-harmonic viscosity
Ht = np.sum(dh)
Ht_b = np.sum(dh_b)
toc = np.array([1.31693E+01,1.81693E+01,2.41693E+01])
bf = delta_ek*f0/(2*dh[-1])

#fréquences Brunt-vaisala

N_12 = g_prime[0]/((dh[0]+dh[1])/2)
N_23 = g_prime[1]/((dh[1]+dh[2])/2)
N2 = np.array([N_12,N_23])

In [3]:
dir_abs  = '/mnt/berger2/equipes/IGE/meom/workdir/berger2/qgcm-data/double_gyre_coupled/outdata_7_dt10/'
dir_rel  = '/mnt/berger2/equipes/IGE/meom/workdir/berger2/qgcm-data/double_gyre_coupled/outdata_6tdiff_dt10/'

file0 = 'ocpo.nc'
file1 = 'ocsst.nc'

In [4]:
#get fields psi
pfile_abs = dir_abs + file0
pfile_rel = dir_rel + file0

toc_file_abs = dir_abs + file1
toc_file_rel = dir_rel + file1


f_abs = netcdf.netcdf_file(dir_abs + file0,'r')
f_rel = netcdf.netcdf_file(dir_rel + file0,'r')


In [35]:
si_t = f_abs.variables['time'][:].copy().size

p = qg.read_qgcm(pfile_abs, 0, var='p', rescale = 1/f0, interp = True, subtract_bc = False )
nz,ny,nx = p.shape

test = si_t

average = True

In [6]:
# flot moyen abs
flot_me = np.load('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test/meom_test/Lorenz_cycle/tableaux_termes_abs/termes_flot_moyen_abs.npz')
u_me_abs = flot_me['u_me']
v_me_abs = flot_me['v_me']

flot_me_bis = np.load('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test/meom_test/Lorenz_cycle/tableaux_termes_abs/etapes_flot_moyen_abs.npz')
         
p_me_abs = flot_me_bis['p_me']
b_me_abs = flot_me_bis['b_me']
w_me_abs = flot_me_bis['w_me']


In [7]:
#flot moyen rel
flot_me = np.load('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test/meom_test/Lorenz_cycle/tableaux_termes_rel/termes_flot_moyen_rel.npz')
u_me_rel = flot_me['u_me']
v_me_rel = flot_me['v_me']

flot_me_bis = np.load('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test/meom_test/Lorenz_cycle/tableaux_termes_rel/etapes_flot_moyen_rel.npz')
         
p_me_rel = flot_me_bis['p_me']
b_me_rel = flot_me_bis['b_me']
w_me_rel = flot_me_bis['w_me']

In [8]:
#uninterp b_me with diff
dbdx_me_abs = np.diff(b_me_abs,1,2)/Delta
dbdy_me_abs = np.diff(b_me_abs,1,1)/Delta
dbdx_me_rel = np.diff(b_me_rel,1,2)/Delta
dbdy_me_rel = np.diff(b_me_rel,1,1)/Delta


#compute the expression sparately with padding to obtain (513,512)ish arrays
dbdx_pad_abs = np.pad(dbdx_me_abs,((0,0),(0,0),(1,1)))
dbdy_pad_abs = np.pad(dbdy_me_abs,((0,0),(1,1),(0,0)))
dbdx_pad_rel = np.pad(dbdx_me_rel,((0,0),(0,0),(1,1)))
dbdy_pad_rel = np.pad(dbdy_me_rel,((0,0),(1,1),(0,0)))

#uninterp u_me and v_me with diff
dudx_me_abs = np.diff(u_me_abs,1,2)/Delta
dudy_me_abs = np.diff(u_me_abs,1,1)/Delta
dvdx_me_abs = np.diff(v_me_abs,1,2)/Delta
dvdy_me_abs = np.diff(v_me_abs,1,1)/Delta

dudx_me_rel = np.diff(u_me_rel,1,2)/Delta
dudy_me_rel = np.diff(u_me_rel,1,1)/Delta
dvdx_me_rel = np.diff(v_me_rel,1,2)/Delta
dvdy_me_rel = np.diff(v_me_rel,1,1)/Delta

#compute the expression sparately with padding to obtain (513,512)ish arrays
dudx_pad_abs = np.pad(dudx_me_abs,((0,0),(0,0),(1,1)))
dudy_pad_abs = np.pad(dudy_me_abs,((0,0),(1,1),(0,0)))
dvdx_pad_abs = np.pad(dvdx_me_abs,((0,0),(0,0),(1,1)))
dvdy_pad_abs = np.pad(dvdy_me_abs,((0,0),(1,1),(0,0)))

dudx_pad_rel = np.pad(dudx_me_rel,((0,0),(0,0),(1,1)))
dudy_pad_rel = np.pad(dudy_me_rel,((0,0),(1,1),(0,0)))
dvdx_pad_rel = np.pad(dvdx_me_rel,((0,0),(0,0),(1,1)))
dvdy_pad_rel = np.pad(dvdy_me_rel,((0,0),(1,1),(0,0)))


In [9]:
print(dvdy_pad_rel.shape)

(3, 513, 512)


In [36]:
%%time

#physical map

mpe2epe_me_abs= np.zeros((nz-1,ny,nx))
epe2mpe_me_abs= np.zeros((nz-1,ny,nx))
mke2eke_me_abs= np.zeros((nz,ny,nx))
eke2mke_me_abs= np.zeros((nz,ny,nx))

mpe2epe_me_rel= np.zeros((nz-1,ny,nx))
epe2mpe_me_rel= np.zeros((nz-1,ny,nx))
mke2eke_me_rel= np.zeros((nz,ny,nx))
eke2mke_me_rel= np.zeros((nz,ny,nx))


n_me=1

for it in range(0,test):
    
    p_abs = qg.read_qgcm(pfile_abs, it, var='p', rescale = 1/f0, interp = True, subtract_bc = True)
    p_rel = qg.read_qgcm(pfile_rel, it, var='p', rescale = 1/f0, interp = True, subtract_bc = True) 
    
    u_abs,v_abs = qg.comp_vel(p_abs, Delta=Delta, loc='center')
    u_rel,v_rel = qg.comp_vel(p_rel, Delta=Delta, loc='center')
    
    b_abs = qg.p2b(p_abs, dh, f0)
    b_rel = qg.p2b(p_rel, dh, f0)
  
    u_p_abs = u_abs - u_me_abs
    v_p_abs = v_abs - v_me_abs
    b_p_abs = b_abs - b_me_abs
    p_p_abs = p_abs - p_me_abs
    
    u_p_rel = u_rel - u_me_rel
    v_p_rel = v_rel - v_me_rel
    b_p_rel = b_rel - b_me_rel
    p_p_rel = p_rel - p_me_rel
        
    u_p_abs_use = reduce_z_dim(u_p_abs, dh)
    v_p_abs_use = reduce_z_dim(v_p_abs, dh)
    u_p_rel_use = reduce_z_dim(u_p_rel, dh)
    v_p_rel_use = reduce_z_dim(v_p_rel, dh)
    
    #MPE to EPE, MPE perspective
    
    #uninterp b with diff
    dbdx_abs = np.diff(b_abs,1,2)/Delta
    dbdy_abs = np.diff(b_abs,1,1)/Delta
    dbdx_rel = np.diff(b_rel,1,2)/Delta
    dbdy_rel = np.diff(b_rel,1,1)/Delta

    #uninterp u and v with average of
    u_unx_abs = 0.5*(u_p_abs_use[:,:,1:] + u_p_abs_use[:,:,:-1])
    v_uny_abs = 0.5*(v_p_abs_use[:,1:,:] + v_p_abs_use[:,:-1,:])
    u_unx_rel = 0.5*(u_p_rel_use[:,:,1:] + u_p_rel_use[:,:,:-1])
    v_uny_rel = 0.5*(v_p_rel_use[:,1:,:] + v_p_rel_use[:,:-1,:])

    #compute the expression sparately with padding to obtain (513,512)ish arrays
    u_dbdx_abs = np.pad(u_unx_abs*dbdx_abs,((0,0),(0,0),(1,1)))
    v_dbdy_abs = np.pad(v_uny_abs*dbdy_abs,((0,0),(1,1),(0,0)))
    u_dbdx_rel = np.pad(u_unx_rel*dbdx_rel,((0,0),(0,0),(1,1)))
    v_dbdy_rel = np.pad(v_uny_rel*dbdy_rel,((0,0),(1,1),(0,0)))


    #compute the expression
    mpe2epe_abs = (-b_me_abs/N2[:,None,None])*0.5*(u_dbdx_abs[:,:,1:] + u_dbdx_abs[:,:,:-1] + v_dbdy_abs[:,1:,:] + v_dbdy_abs[:,:-1,:])
    mpe2epe_rel = (-b_me_rel/N2[:,None,None])*0.5*(u_dbdx_rel[:,:,1:] + u_dbdx_rel[:,:,:-1] + v_dbdy_rel[:,1:,:] + v_dbdy_rel[:,:-1,:])
    
    #MPE to EPE, EPE perspective
    
    
    epe2mpe_abs = (0.5/N2[:,None,None])*((dbdx_pad_abs[:,:,1:] + dbdx_pad_abs[:,:,:-1])*u_p_abs_use*b_p_abs
                + (dbdy_pad_abs[:,:-1,:]+dbdy_pad_abs[:,1:,:])*v_p_abs_use*b_p_abs)
    
    
    
    epe2mpe_rel = (0.5/N2[:,None,None])*((dbdx_pad_rel[:,:,1:] + dbdx_pad_rel[:,:,:-1])*u_p_rel_use*b_p_rel
                + (dbdy_pad_rel[:,:-1,:]+dbdy_pad_rel[:,1:,:])*v_p_rel_use*b_p_rel)
    
    #MKE to EKE, EKE perspective
    #uninterp u_p and v_p products with diff
    du2_dx_abs = np.diff(u_p_abs**2, 1,2)/Delta
    duv_dy_abs = np.diff(u_p_abs*v_p_abs,1,1)/Delta
    duv_dx_abs = np.diff(u_p_abs*v_p_abs,1,2)/Delta
    dv2_dy_abs = np.diff(v_p_abs**2, 1,1)/Delta
    
    du2_dx_rel = np.diff(u_p_rel**2, 1,2)/Delta
    duv_dy_rel = np.diff(u_p_rel*v_p_rel,1,1)/Delta
    duv_dx_rel = np.diff(u_p_rel*v_p_rel,1,2)/Delta
    dv2_dy_rel = np.diff(v_p_rel**2, 1,1)/Delta

    #compute the expression sparately with padding to obtain (513,512)ish arrays
    du2_dx_pad_abs = np.pad(du2_dx_abs,((0,0),(0,0),(1,1)))
    duv_dy_pad_abs = np.pad(duv_dy_abs,((0,0),(1,1),(0,0)))
    duv_dx_pad_abs = np.pad(duv_dx_abs,((0,0),(0,0),(1,1)))
    dv2_dy_pad_abs = np.pad(dv2_dy_abs,((0,0),(1,1),(0,0)))
    
    du2_dx_pad_rel = np.pad(du2_dx_rel,((0,0),(0,0),(1,1)))
    duv_dy_pad_rel = np.pad(duv_dy_rel,((0,0),(1,1),(0,0)))
    duv_dx_pad_rel = np.pad(duv_dx_rel,((0,0),(0,0),(1,1)))
    dv2_dy_pad_rel = np.pad(dv2_dy_rel,((0,0),(1,1),(0,0)))

    eke2mke_abs = 0.5*((du2_dx_pad_abs[:,:,1:]+du2_dx_pad_abs[:,:,:-1])*u_me_abs 
              +(duv_dy_pad_abs[:,1:,:]+duv_dy_pad_abs[:,:-1,:])*u_me_abs
              +(duv_dx_pad_abs[:,:,1:]+duv_dx_pad_abs[:,:,:-1])*v_me_abs
              +(dv2_dy_pad_abs[:,1:,:]+dv2_dy_pad_abs[:,:-1,:])*v_me_abs)
    
    eke2mke_rel = 0.5*((du2_dx_pad_rel[:,:,1:]+du2_dx_pad_rel[:,:,:-1])*u_me_rel 
          +(duv_dy_pad_rel[:,1:,:]+duv_dy_pad_rel[:,:-1,:])*u_me_rel
          +(duv_dx_pad_rel[:,:,1:]+duv_dx_pad_rel[:,:,:-1])*v_me_rel
          +(dv2_dy_pad_rel[:,1:,:]+dv2_dy_pad_rel[:,:-1,:])*v_me_rel)
    
    #MKE to EKE, MKE perspective (à checker la perspective)
    mke2eke_abs = 0.5*((dudx_pad_abs[:,:,1:]+dudx_pad_abs[:,:,:-1])*u_p_abs**2 
          +(dudy_pad_abs[:,1:,:]+dudy_pad_abs[:,:-1,:])*u_p_abs*v_p_abs
          +(dvdx_pad_abs[:,:,1:]+dvdx_pad_abs[:,:,:-1])*u_p_abs*v_p_abs
          +(dvdy_pad_abs[:,1:,:]+dvdy_pad_abs[:,:-1,:])*v_p_abs**2)
    
    mke2eke_rel = 0.5*((dudx_pad_rel[:,:,1:]+dudx_pad_rel[:,:,:-1])*u_p_rel**2 
          +(dudy_pad_rel[:,1:,:]+dudy_pad_rel[:,:-1,:])*u_p_rel*v_p_rel
          +(dvdx_pad_rel[:,:,1:]+dvdx_pad_rel[:,:,:-1])*u_p_rel*v_p_rel
          +(dvdy_pad_rel[:,1:,:]+dvdy_pad_rel[:,:-1,:])*v_p_rel**2)
   
    
    mpe2epe_me_abs += (mpe2epe_abs - mpe2epe_me_abs)/n_me 
    epe2mpe_me_abs += (epe2mpe_abs - epe2mpe_me_abs)/n_me 
    mke2eke_me_abs += (mke2eke_abs - mke2eke_me_abs)/n_me 
    eke2mke_me_abs += (eke2mke_abs - eke2mke_me_abs)/n_me
    
    mpe2epe_me_rel += (mpe2epe_rel - mpe2epe_me_rel)/n_me 
    epe2mpe_me_rel += (epe2mpe_rel - epe2mpe_me_rel)/n_me 
    mke2eke_me_rel += (mke2eke_rel - mke2eke_me_rel)/n_me 
    eke2mke_me_rel += (eke2mke_rel - eke2mke_me_rel)/n_me
    
    n_me += 1
    


CPU times: user 4min 1s, sys: 2.88 s, total: 4min 4s
Wall time: 8min 38s


In [ ]:
w_b_me_abs= np.zeros((nl-1,N,naux))
p_w_me_abs= np.zeros((nl,N,naux))

w_b_me_rel= np.zeros((nl-1,N,naux))
p_w_me_rel= np.zeros((nl,N,naux))


n_me=1

for it in range(0,test):
    
    p_abs = qg.read_qgcm(pfile_abs, it, var='p', rescale = 1/f0, interp = True, subtract_bc = True)
    p_rel = qg.read_qgcm(pfile_rel, it, var='p', rescale = 1/f0, interp = True, subtract_bc = True) 
    
    loc_forcing_z_abs = qg.read_qgcm(toc_file_abs, it, 'wekt', rescale=f0/dh[0], interp=False)
    loc_forcing_z_rel = qg.read_qgcm(toc_file_rel, it, 'wekt', rescale=f0/dh[0], interp=False)
    
    loc_forcing_b_abs = qg.read_qgcm(pfile_abs, it, var='entoc', interp = True)
    loc_forcing_b_rel = qg.read_qgcm(pfile_rel, it, var='entoc', interp = True)

    w_abs = qg.get_w(p_abs,dh, N2,f0, Delta, bf,loc_forcing_z_abs, loc_forcing_b_abs, nu=0, nu4=nu4)
    w_rel = qg.get_w(p_rel,dh, N2,f0, Delta, bf,loc_forcing_z_rel, loc_forcing_b_rel, nu=0, nu4=nu4)
    
    b_abs = qg.p2b(p_abs, dh, f0)
    b_rel = qg.p2b(p_rel, dh, f0)
  
    w_p_abs = w_abs - w_me_abs
    b_p_abs = b_abs - b_me_abs
    p_p_abs = p_abs - p_me_abs
    
    w_p_rel = w_rel - w_me_rel
    b_p_rel = b_rel - b_me_rel
    p_p_rel = p_rel - p_me_rel
    
    w_p_abs_use = np.concatenate((zero,w_p_abs,zero), axis=0)
    w_p_rel_use = np.concatenate((zero,w_p_rel,zero), axis=0)
        
    w_b_p_abs = w_p_abs*b_p_abs
    p_w_p_abs = p_p_abs*div_z(w_p_abs_use,dh,f0)
    w_b_p_rel = w_p_rel*b_p_rel
    p_w_p_rel = p_p_rel*div_z(w_p_rel_use,dh,f0)
    
    
    w_b_me_abs += (w_b_p_abs - w_b_me_abs)/n_me
    p_w_me_abs += (p_w_p_abs - p_w_me_abs)/n_me
    w_b_me_rel += (w_b_p_rel - w_b_me_rel)/n_me
    p_w_me_rel += (p_w_p_rel - p_w_me_rel)/n_me
    
    n_me += 1
    


In [37]:
np.savez('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test//meom_test/Lorenz_cycle/cycle_termes_sep/tableaux_termes_exch/termes_abs.npz',
    mpe2epe_me_abs = mpe2epe_me_abs,
    epe2mpe_me_abs = epe2mpe_me_abs,
    mke2eke_me_abs = mke2eke_me_abs, 
    eke2mke_me_abs = eke2mke_me_abs)
    
    


In [38]:
np.savez('/mnt/berger2/equipes/IGE/meom/workdir/berger2/git/meom_test//meom_test/Lorenz_cycle/cycle_termes_sep/tableaux_termes_exch/termes_rel.npz',
    mpe2epe_me_rel = mpe2epe_me_rel,
    epe2mpe_me_rel = epe2mpe_me_rel, 
    mke2eke_me_rel = mke2eke_me_rel,
    eke2mke_me_rel = eke2mke_me_rel)